# Importing all the required libraries

In [1]:
#Importing all the required libraries

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
from collections import defaultdict
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# Preprocessing

In [0]:
df_train= pd.read_csv('/content/hm_train.csv')
df_test= pd.read_csv('/content/hm_test.csv')

# Preprocessing the data
df_train['cleaned_hm'] = df_train['cleaned_hm'].apply(lambda x: x.replace('\n',''))
df_train['cleaned_hm'] = df_train['cleaned_hm'].apply(lambda x: x.replace('\r',''))

cleaned = defaultdict(list)
for hm in df_train['cleaned_hm']:
    g = sent_tokenize(hm)
    if len(g) > 1:
        for i in range(len(g)):
            cleaned['cleaned_hm'].append(g[i])
            
            b=int(re.search(r'\d+', str(df_train[df_train['cleaned_hm'] == hm]['predicted_category'])).group())
            cleaned['predicted_category'].append(df_train[df_train['cleaned_hm'] == hm]['predicted_category'][b])
    else:
        cleaned['cleaned_hm'].append(hm)
        b=int(re.search(r'\d+', str(df_train[df_train['cleaned_hm'] == hm]['predicted_category'])).group())
        cleaned['predicted_category'].append(df_train[df_train['cleaned_hm'] == hm]['predicted_category'][b])

cleaned_df = pd.DataFrame(cleaned)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(cleaned_df['cleaned_hm'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

df_test['cleaned_hm'] = df_test['cleaned_hm'].apply(lambda x: x.replace('\n',''))
df_test['cleaned_hm'] = df_test['cleaned_hm'].apply(lambda x: x.replace('\r',''))

# Building model

In [5]:
param = {'loss':('squared_hinge', 'hinge'),
         }
from sklearn.model_selection import StratifiedShuffleSplit
n_samples = cleaned_df.shape[0]
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.3, random_state=0)

svc = LinearSVC()

clf = GridSearchCV(svc,param_grid=param, cv=cv)
clf.fit(X_train_tfidf, cleaned_df['predicted_category'])
g= clf.predict(count_vect.transform(df_test['cleaned_hm']))

result= pd.DataFrame()

result['hmid'] = df_test['hmid']
result['predicted_category'] =g

result.to_csv('cln_lsvm_gs1.csv')

result.head()

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


,hmid,predicted_category
0,88305,bonding
1,88306,affection
2,88307,affection
3,88308,bonding
4,88309,affection
